In [1]:
import numpy as np
import math



In [43]:

document1 = "Machine learning teaches machine how to learn"
document2 = "i am apple"
document3 = "mango i like learning"
document4 = "learning is shit"
docs=[document1,document2,document3,document4]

def termFrequency(term, document):
    normalizeDocument = document.lower().split()
    # print(normalizeDocument)
    return normalizeDocument.count(term.lower()) / float(len(normalizeDocument))
   
# IDF of a term
def inverseDocumentFrequency(term, documents):
    count = 0
    for doc in documents:
        if term.lower() in doc.lower().split():
            # print(doc.lower().split())

            count += 1
    if count > 0:
        # print(len(documents),count,float(len(documents))/count,math.log(float(len(documents))/count,10))

        return 1.0 + math.log(float(len(documents))/count,10)

    else:
        return 1.0
        
# tf-idf of a term in a document
def tf_idf(term, document, documents):
    tf = termFrequency(term, document)
    idf = inverseDocumentFrequency(term, documents)
    # print(tf,idf)
    return tf*idf
for i in docs:
    print("---")
    print(tf_idf("learning",i,docs))


---
0.16070553380118568
---
0.0
---
0.281234684152075
---
0.3749795788694333


In [106]:
import torch
from transformers import BertModel, BertTokenizer
from rank_bm25 import BM25Okapi

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

In [119]:
# documents1221=[]
# bm25=BM25Okapi()
def tokenize_documents( documents):
    """Tokenize and prepare documents for BM25."""
    tokenized_docs = [tokenizer.tokenize(doc) for doc in documents]
    # documents1221 = tokenized_docs
    bm25 = BM25Okapi(tokenized_docs)
    # print(tokenized_docs)

# def predict_term_weights(self, query):
#     """Predict term weights for the given query."""
#     # Tokenize query
#     query_tokens = self.tokenizer.tokenize(query)
#     query_ids = self.tokenizer.convert_tokens_to_ids(query_tokens)
    # print(bm25.doc_freqs)
    return bm25

def predict_term_weights( query):
    """Predict term weights for the given query using BERT embeddings."""
    inputs = tokenizer(query, return_tensors="pt")
    # print(inputs)
    with torch.no_grad():
        outputs = model(**inputs)
        # Use the last hidden state
        last_hidden_states = outputs.last_hidden_state
    # print(len(last_hidden_states[0]))
    # print((last_hidden_states))
    # Simplified weight prediction logic: Use the norm of the embeddings as weights
    # This is a placeholder and should be replaced with a proper mechanism
    weights = torch.norm(last_hidden_states, dim=-1).squeeze().tolist()
    # print(weights)

    # Associate weights with tokens. This assumes no special tokens (CLS, SEP) for simplicity.
    tokens = tokenizer.convert_ids_to_tokens(inputs["input_ids"].squeeze().tolist())
    return dict(zip(tokens, weights))
# print("")

In [120]:
# for i in docs:
    # documents1221 = tokenized_docs
tokenized_docs = [tokenizer.tokenize(doc) for doc in docs]
    #
bm25 = BM25Okapi(tokenized_docs)
predict_term_weights("Machine learning teaches machine how to learn")

{'[CLS]': 14.703083038330078,
 'machine': 15.307113647460938,
 'learning': 14.563008308410645,
 'teaches': 14.735929489135742,
 'how': 16.826915740966797,
 'to': 15.747702598571777,
 'learn': 15.463006019592285,
 '[SEP]': 15.313431739807129}

In [149]:

from collections import defaultdict
def search( query):
    """Perform a search by optimizing query-document matching scores."""
    term_weights = predict_term_weights(query)
    weighted_query = [(term, weight) for term, weight in term_weights.items()]
 

    # # Calculate scores using BM25 + term weights (simplified version)
    scores = defaultdict(float)
    # print(scores.items())
    # # print("sssss",weighted_query)
    
    for term, weight in weighted_query:
        term_scores = bm25.get_scores([term])
        # print(term_scores)
        for doc_id, score in enumerate(term_scores):
            # print(doc_id,score)
            scores[doc_id] += score * weight
        # print(scores.items())
    # # Sort documents by their score
        sorted_docs = sorted(scores.items(), key=lambda x: x[1], reverse=True)
        return [docs[doc_id] for doc_id, _ in sorted_docs]


In [151]:
search("i like mango")

['Machine learning teaches machine how to learn',
 'i am apple',
 'mango i like learning',
 'learning is shit']

In [40]:
# for i in docs:
print(termFrequency("machine",document2))


0.0


In [54]:
documents = [  
    "Machine learning is a subset of artificial intelligence.",
    "Deep learning is a type of machine learning.",
    "Natural language processing is a part of machine learning.",
    "Machine learning algorithms improve over time through learning from data."]
for term in document1.split():
    print("{}: {}".format(term, inverseDocumentFrequency(term, documents)))
 
print("---------")
# tf-idf test run:
documents = [document1, document2, document3]
for term in document1.split(" "):
    print("{}: {}".format(term, tf_idf(term, document1, documents)))

Machine: 1.0
learning: 1.2876820724517808
teaches: 1.0
machine: 1.0
how: 1.0
to: 1.0
learn: 1.0
---------
['machine', 'learning', 'teaches', 'machine', 'how', 'to', 'learn']
Machine: 0.5996035110480314
['machine', 'learning', 'teaches', 'machine', 'how', 'to', 'learn']
learning: 0.2998017555240157
['machine', 'learning', 'teaches', 'machine', 'how', 'to', 'learn']
teaches: 0.2998017555240157
['machine', 'learning', 'teaches', 'machine', 'how', 'to', 'learn']
machine: 0.5996035110480314
['machine', 'learning', 'teaches', 'machine', 'how', 'to', 'learn']
how: 0.2998017555240157
['machine', 'learning', 'teaches', 'machine', 'how', 'to', 'learn']
to: 0.2998017555240157
['machine', 'learning', 'teaches', 'machine', 'how', 'to', 'learn']
learn: 0.2998017555240157


In [1]:
import nltk
import pandas as pd
import sklearn as sk
import math 

nltk.download('stopwords')
from nltk.corpus import stopwords


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/praveenlawyantra/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
first_sentence = "Data Science is the sexiest job of the 21st century"
second_sentence = "machine learning is the key for data science"
#split so each word have their own string
first_sentence = first_sentence.split(" ")
second_sentence = second_sentence.split(" ")#join them to remove common duplicate words
total= set(first_sentence).union(set(second_sentence))
print(total)

stop_words = set(stopwords.words('english'))
# stop_words


{'data', 'is', 'Science', 'the', 'machine', 'of', 'job', 'key', 'for', 'learning', 'science', 'Data', '21st', 'sexiest', 'century'}


In [5]:
wordDictA = dict.fromkeys(total, 0) 
wordDictB = dict.fromkeys(total, 0)
for word in first_sentence:
    wordDictA[word]+=1
    
for word in second_sentence:
    wordDictB[word]+=1

filtered_sentence = [w for w in wordDictA if not w in stop_words]
print(filtered_sentence)
pd.DataFrame([wordDictA, wordDictB])


['data', 'Science', 'machine', 'job', 'key', 'learning', 'science', 'Data', '21st', 'sexiest', 'century']


,data,is,Science,the,machine,of,job,key,for,learning,science,Data,21st,sexiest,century
0,0,1,1,2,0,1,1,0,0,0,0,1,1,1,1
1,1,1,0,1,1,0,0,1,1,1,1,0,0,0,0


In [64]:
def computeTF(wordDict, doc):
    tfDict = {}
    corpusCount = len(doc)
    for word, count in wordDict.items():
        tfDict[word] = count/float(corpusCount)
    return(tfDict)
#running our sentences through the tf function:
tfFirst = computeTF(wordDictA, first_sentence)
tfSecond = computeTF(wordDictB, second_sentence)
#Converting to dataframe for visualization
tf = pd.DataFrame([tfFirst, tfSecond])
tf

,sexiest,is,Science,job,century,science,for,data,Data,key,of,learning,machine,21st,the
0,0.1,0.100,0.1,0.1,0.1,0.000,0.000,0.000,0.1,0.000,0.1,0.000,0.000,0.1,0.200
1,0.0,0.125,0.0,0.0,0.0,0.125,0.125,0.125,0.0,0.125,0.0,0.125,0.125,0.0,0.125


In [65]:
def computeIDF(docList):
    idfDict = {}
    N = len(docList)
    
    idfDict = dict.fromkeys(docList[0].keys(), 0)
    for word, val in idfDict.items():
        idfDict[word] = math.log10(N / (float(val) + 1))
        
    return(idfDict)
#inputing our sentences in the log file
idfs = computeIDF([wordDictA, wordDictB])

In [66]:
def computeTFIDF(tfBow, idfs):
    tfidf = {}
    for word, val in tfBow.items():
        tfidf[word] = val*idfs[word]
    return(tfidf)
#running our two sentences through the IDF:
idfFirst = computeTFIDF(tfFirst, idfs)
idfSecond = computeTFIDF(tfSecond, idfs)
#putting it in a dataframe
idf= pd.DataFrame([idfFirst, idfSecond])
print(idf)

    sexiest        is   Science       job   century   science       for   
0  0.030103  0.030103  0.030103  0.030103  0.030103  0.000000  0.000000  \
1  0.000000  0.037629  0.000000  0.000000  0.000000  0.037629  0.037629   

       data      Data       key        of  learning   machine      21st   
0  0.000000  0.030103  0.000000  0.030103  0.000000  0.000000  0.030103  \
1  0.037629  0.000000  0.037629  0.000000  0.037629  0.037629  0.000000   

        the  
0  0.060206  
1  0.037629  
